In [1]:
!pip install cplex --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host=files.pythonhosted.org

!pip install docplex --trusted-host pypi.org --trusted-host pypi.python.org --trusted-host=files.pythonhosted.org

In [13]:
import sys
import docplex.mp as mp
from docplex.mp.model import Model

In [25]:
class LP_Project: 
    def __init__(self):
        self.model = Model(name='Candy Mix forIMSE Candy Supply')
            
    def set_var(self):
            #decision variable that dictates the quantity of door Sppky Surprise to produce
        self.SpookyP1 = self.model.continuous_var(name = 'Pounds of Spooky Surprise at Plant 1 ',lb= 0)
        self.SpookyP2 = self.model.continuous_var(name = 'Pounds of Spooky Surprise at Plant 2 ',lb = 0)
        self.SpookyP3 = self.model.continuous_var(name = 'Pounds of Spooky Surprise at Plant 3 ',lb = 0)
            # decision variable that dictates the quantity of door Fruity Fun to produce
        self.FruityP1 = self.model.continuous_var(name = 'Pounds of Fruity Fun at Plant 1 ',lb = 0)
        self.FruityP2 = self.model.continuous_var(name = 'Pounds of Fruity Fun at Plant 2 ',lb = 0)
        self.FruityP3 = self.model.continuous_var(name = 'Pounds of Fruity Fun at Plant 3 ',lb = 0)
         # decision variable that dictates the quantity of door Chocolate Craze to produce
        self.ChocolateP1 = self.model.continuous_var(name = 'Pounds of Chocolate Craze at Plant 1 ',lb = 0)
        self.ChocolateP2 = self.model.continuous_var(name = 'Pounds of Chocolate Craze at Plant 2 ',lb= 0)
        self.ChocolateP3 = self.model.continuous_var(name = 'Pounds of Chocolate Craze at Plant 3 ',lb = 0)
         # decision variable that dictates the quantity of door Tis the Season to produce
        self.TisP1 = self.model.continuous_var(name = 'Pounds of Tis the Season at Plant 1 ',lb = 0)
        self.TisP2 = self.model.continuous_var(name = 'Pounds of Tis the Season at Plant 2 ',lb = 0)
        self.TisP3 = self.model.continuous_var(name = 'Pounds of Tis the Season at Plant 3 ',lb = 0)
        
    def set_objective(self):
            #set profit portion of obj function
        profit=((10*(self.SpookyP1+self.SpookyP2))+(8*(self.FruityP1+self.FruityP2+self.FruityP3))+
                (14*(self.ChocolateP2+self.ChocolateP3)+(18*(self.TisP1+self.TisP3))))
            #set cost portion of obj function
        cost=((-5.2*((0.4*self.SpookyP1)+(0.7*self.FruityP1)+(3.1*self.TisP1)))+
              (-5.4*((0.6*self.SpookyP2)+(0.9*self.FruityP2)+(2.7*self.ChocolateP2)))+
              (-5.3*((0.5*self.FruityP3)+(2.5*self.ChocolateP3)+(2.9*self.TisP3))))
            
        self.model.maximize(profit+cost)
            
    
    def set_constraints(self):
            #Constraints relating to Labor hours Avalible at each plant
        # C1: labor hour avalibility at Plant 1
        self.model.add_constraint((0.4*self.SpookyP1)+(0.7*self.FruityP1)+(3.1*self.TisP1) <= 3000) 
        
        # C2: Labor hours of Plant 2
        self.model.add_constraint((0.6*self.SpookyP2)+(0.9*self.FruityP2)+(2.7*self.ChocolateP2)<= 4800)
        
        #C3:Labor Hours of Plant 3
        self.model.add_constraint((0.5*self.FruityP3)+(2.5*self.ChocolateP3)+(2.9*self.TisP3)<= 3200)
            
            #Constraints relating to max capacity of each candy
        #C4:Capacity constraint of Spooky Surpirse
        self.model.add_constraint((self.SpookyP1+self.SpookyP2)<= 3250)
        
        #C5: Capacity constraint of Fruity Fun
        self.model.add_constraint((self.FruityP1+self.FruityP2+self.FruityP3)<= 2500)
        
        #C6:Capacity constraint of Chocolate Craze
        self.model.add_constraint((self.ChocolateP2+self.ChocolateP3)<= 5000)
        
        #C7:Capacity constraint of Tis the Season 
        self.model.add_constraint((self.TisP1+self.TisP3)<= 3000)
        
                #50% Capacity Constraints 
        #C8-9:Ensuring no more than 50% of Spooky Season is being produced in either Plant 1 or Plant 2
        self.model.add_constraint(((.5*self.SpookyP2)-(0.5*self.SpookyP1))>=0)
        self.model.add_constraint(((.5*self.SpookyP1)-(0.5*self.SpookyP2))>=0)
        
        #C10-12:Ensuring no more than 50% of Fruity Fun is being produced in any Plant 
        self.model.add_constraint(((.5*self.FruityP2)+(0.5*self.FruityP3)-(0.5*self.FruityP1))>=0)
        self.model.add_constraint(((.5*self.FruityP1)+(0.5*self.FruityP3)-(0.5*self.FruityP2))>=0)
        self.model.add_constraint(((.5*self.FruityP1)+(0.5*self.FruityP2)-(0.5*self.FruityP3))>=0)
        
        
        #C33-14:Ensuring no more than 50% of Chocolate Craze is being produced in either Plant 3 or Plant 2
        self.model.add_constraint(((.5*self.ChocolateP3)-(0.5*self.ChocolateP2))>=0)
        self.model.add_constraint(((.5*self.ChocolateP2)-(0.5*self.ChocolateP3))>=0)
        
        #C15-16:Ensuring no more than 50% of Tis the Season is being produced in either Plant 1 or Plant 3
        self.model.add_constraint(((.5*self.TisP3)-(0.5*self.TisP1))>=0)
        self.model.add_constraint(((.5*self.TisP1)-(0.5*self.TisP3))>=0)

        # C17: Constraing setting Minimum requirement of Chocolate Crazr
        self.model.add_constraint((self.ChocolateP2+self.ChocolateP3)>= 1500)
        
    def print_shadow_prices(self):
        # Shadow prices for the constraints
        shadow_prices = self.model.dual_values(self.model.iter_constraints())
        
        # print the shadow prices
        for y, constraint in enumerate(self.model.iter_constraints()):
            print(f"Shadow Price for Constraint {y + 1}: {shadow_prices[y]}")    
        
    # How the solver is called
    def solve_model(self):
        # solve the problem if possible
        solution = self.model.solve()           
        if solution:
            print(self.model.solution)
        else:
            #if not possible, print:
            print("No solution found.")
            return None

In [26]:
#Calling the Solver
solver = LP_Project()
solver.set_var()
solver.set_objective()
solver.set_constraints()
solver.solve_model()
solver.print_shadow_prices()

solution for: Candy Mix forIMSE Candy Supply
objective: 37208.6
status: OPTIMAL_SOLUTION(2)
Pounds of Spooky Surprise at Plant 1 =1625.000
Pounds of Spooky Surprise at Plant 2 =1625.000
Pounds of Fruity Fun at Plant 1 =1250.000
Pounds of Fruity Fun at Plant 3 =1250.000
Pounds of Chocolate Craze at Plant 2 =750.000
Pounds of Chocolate Craze at Plant 3 =750.000
Pounds of Tis the Season at Plant 1 =241.379
Pounds of Tis the Season at Plant 3 =241.379

Shadow Price for Constraint 1: 0
Shadow Price for Constraint 2: 0
Shadow Price for Constraint 3: 1.5551724137931036
Shadow Price for Constraint 4: 7.34
Shadow Price for Constraint 5: 4.466206896551724
Shadow Price for Constraint 6: 0
Shadow Price for Constraint 7: 0
Shadow Price for Constraint 8: -1.1600000000000001
Shadow Price for Constraint 9: 0
Shadow Price for Constraint 10: 0
Shadow Price for Constraint 11: 0
Shadow Price for Constraint 12: -0.21241379310344755
Shadow Price for Constraint 13: -2.557931034482757
Shadow Price for Constra